# Find the starting lattice parameters using MILK

Until now in this tutorial we have focused on a general problem.
Now we will apply what we have done to a Rietveld analysis.

The key concepts you should gain from this page:

 * How to create a surrogate model of an expensive cost function and use the surrogate model to guide the optimization.
 * How to call an optimization with different multi-processing pools.
 
**Note, you are going to need the data and ``template.par`` from the [MILK XRD tutorial](https://github.com/lanl/MILK/wiki/XRD-Tutorial).
If running from the notebook, you will need this data directory and parameter file in the same location as you run the notebook.**

**Note, this part of the tutorial doesn't run robstly in iPython notebook on every machine. I suggest running it from a script instead of the notebook itself.**

## Define cost function using MILK

In our prior problems, we defined a cost function to optimize which was a simple Python function.
For using a program like MAUD which writes files to disk we need to make sure each optimizer has its own directory set up for itself.
Otherwise different optimizers would be overwriting each other in the same directory.

Therefore, we are going to create a slightly more complicated cost function using the ``AbstractFunction`` class from Mystic.
The analagous function to our prior cost functions is ``CostFunction.function`` below.
Our ``CostFunction.function`` calls ``CostFunction.initialize`` and ``CostFunction.cost_function``.

MILK will want to setup a directory with files that are read by MAUD.
This is only called once at the beginning.
Therefore, there is an ``CostFunciton.initialize`` function which is called once at the very beginning of an optimization.
This ``initialize`` simply sets up the directory (e.g. copy needed files).
We name the directory after the processor's name.

Then there is the ``CostFunction.cost_function`` which sets the lattice parameters using MILK, does a refinement step, and returns the R-factor.
This is called for every parameter set in the optimization.

**Note, you will need to make one change below.**
You will need to change the path to the ``template.par`` file that is being copied below.
It will exist from the MILK tutorial.
You'd really want to pass this as an attribute on the class but for the sake of brevity we will edit the line here in place.

In [ ]:
import MILK
import multiprocess
import os
import shutil
import time
from mystic import models
from spotlight import filesystem

class CostFunction(models.AbstractFunction):

    def __init__(self, ndim=None, config=None):
        super().__init__(ndim=ndim)
        self.initialized = False
        self.config = config
        
    def reset(self):
        self.initialized = False

    def function(self, p):
        if not self.initialized:
            self.initialize()
            self.initialized = True
        return self.cost_function(p)

    def initialize(self):

        # get editor and maudText
        self.editor = MILK.parameterEditor.editor()
        self.editor.parseConfig(config)
        self.maudText = MILK.maud.maudText()
        self.maudText.parseConfig(config)

        # set run dir based on process name
        self.editor.run_dirs = f"opt_{multiprocess.current_process().name}"
        self.maudText.run_dirs = self.editor.run_dirs

        # create run dir
        if os.path.exists(self.editor.run_dirs):
            shutil.rmtree(self.editor.run_dirs)
        filesystem.mkdir(self.editor.run_dirs)
        filesystem.cp(["/Users/cmbiwer/projects/milk_workshop/sequential_refinement/templates/template_4768.par"], dest=self.editor.run_dirs) # CHANGE THIS LINE

        # set initial phase fractions
        self.editor.set_val(key='_pd_phase_atom_', value='0.33')
        self.editor.free(key='_pd_phase_atom_', wild=[0])

    def cost_function(self, p):

        t0 = time.time()

        # set lattice parameters as values from optimization
        self.editor.set_val(key='cell_length_a', sobj="alpha", value=str(p[0]))
        self.editor.set_val(key='cell_length_c', sobj="alpha", value=str(p[1]))
        self.editor.set_val(key='cell_length_a', sobj="steel", value=str(p[2]))
        self.editor.set_val(key='cell_length_a', sobj="beta", value=str(p[3]))

        # refine
        self.maudText.refinement(itr='1', ifile=self.editor.ifile, ofile=self.editor.ofile, simple_call=True)

        # get the statistic to return to the optimizer
        self.editor.get_val(key="_refine_ls_wR_factor_all")
        stat = float(self.editor.value[0])

        print(f"Our R-factor is {stat} and it took {time.time() - t0}s to compute")

        return stat

## Configure MILK

We will also need to configure MILK for our refinement.
The following is an example of setting MILK to use the [MILK XRD Tutorial dataset](https://github.com/lanl/MILK/wiki/XRD-Tutorial).
For more documentation on the settings of these parameters, see the MILK tutorial.

In [ ]:
# set MILK configuration
config = {"folders": {"work_dir": "",
                      "run_dirs": "ack(wild)",
                      "sub_dir": "",
                      "wild": [0],
                      "wild_range": [[]],
          },
          "compute": {"maud_path": "",
                      "n_maud": 1,
                      "java_opt": "Xmx8G",
                      "clean_old_step_data": False,
                      "cur_step": 1,
                      "log_consol": False,
          },
          "ins": {"riet_analysis_file": "template_4768.par",
                  "riet_analysis_fileToSave": "output.par",
                  "section_title": "Ti64_test_data",
                  "analysis_iteration_number": 4,
                  "LCLS2_detector_config_file": "",
                  "LCLS2_Cspad0_original_image": "",
                  "LCLS2_Cspad0_dark_image": "",
                  "output_plot2D_filename": "plot_",
                  "output_summed_data_filename": "all_spectra",
                  "maud_output_plot_filename": "plot1d_",
                  "output_PF_filename": "PF_",
                  "output_PF": "",
                  "append_simple_result_to": "tmp_simple_results.txt",
                  "append_result_to": "tmp_results.txt",
                  "import_phase": [],
                  "ins_file_name": "MAUDText.ins",
                  "maud_remove_all_datafiles": True,
                  "verbose": 0,
          },
          "interface": {"verbose": 0,
          },
}

## Finding the starting lattice parameters of a Rietveld analysis by optimizing a surrogate model

In the prior page of the tutorial, we went over how to find the global minimum of using a surrogate model.
Recall, that we constructed an interpolated model using the ``InterpModel`` and then minimized until our surrogate and cost function reached <0.001 difference.

Applying this to Rietveld analysis, the global minimum is the best fit parameters for our data.
Below, we follow the same pattern using ``InterpModel`` to create an interpolated surrogate model.
In the ``while`` loop we continuously update the surrogate model, find the minimum of the surrogate model, then evaluate that minimum using our actual cost function (``CostFunction.function`` above) using MILK, and continue until we are below a threshold of agreement between the surrogate model and the actual cost function.

Recall, we introduced this approach because we were able to find a minimum with fewer calls to the actual cost function.
In this case, a call to MILK may take several seconds and in an optimization algorithm calling that hundreds or thousands of times, the time can be hours.

In [ ]:
from mystic import tools
from mystic.solvers import diffev2
from mystic.math.legacydata import dataset, datapoint
from spotlight.bridge.ouq_models import WrapModel
from spotlight.bridge.ouq_models import InterpModel

# set random seed so we can reproduce results
tools.random_seed(0)

# set bounds for parameters to be +/-5%
target = [2.9306538, 4.6817646, 3.6026807, 3.233392]
lower_bounds = [x * 0.95 for x in target]
upper_bounds = [x * 1.05 for x in target]

# remove prior cached results
if os.path.exists("surrogate"):
    shutil.rmtree("surrogate")
        
# generate a sampled dataset for the model
truth = WrapModel("surrogate", CostFunction(4), nx=4, ny=None, cached=False)
bounds = list(zip(lower_bounds, upper_bounds))
data = truth.sample(bounds, pts=[2, 1, 1, 1])

# create surrogate model
surrogate = InterpModel("surrogate", nx=4, ny=None, data=truth, smooth=0.0, noise=0.0,
                        method="thin_plate", extrap=False)

# go until error < 1e-3
error = float("inf")
sign = 1.0
while error > 1e-3:

    # fit surrogate data
    surrogate.fit(data=data)

    # find minimum/maximum of surrogate
    results = diffev2(lambda x: sign * surrogate(x), bounds, npop=20,
                      bounds=bounds, gtol=500, full_output=True)

    # get minimum/maximum of actual expensive model
    xnew = results[0].tolist()
    ynew = truth(xnew)

    # compute error which is actual model value - surrogate model value
    ysur = results[1]
    error = abs(ynew - ysur)

    # print statements
    print("truth", xnew, ynew)
    print("surrogate", xnew, ysur)
    print("error", ynew - ysur, error)
    print("data", len(data))

    # add latest evaulated point with actual expensive model to be used by surrogate in fitting
    pt = datapoint(xnew, value=ynew)
    data.append(pt)

# print the best parameters
print(f"The best solution is {xnew} with Rwp {ynew}")
print(f"The reference solutions is {target}")
ratios = [x / y for x, y in zip(target, xnew)]
print(f"The ratios of to the reference values are {ratios}")

## An ensemble using only the cost function

As state above, the call to MILK may take several seconds.
Below, we present an example of using an ensemble of optimizers in parallel with MILK to find the global minimum.
**Note, this will take awhile. It depends on the number of processors available on your machine.**
We set the number of function calls to 50 to limit the amount of time it will take.

In [ ]:
from mystic.solvers import LatticeSolver
from mystic.solvers import NelderMeadSimplexSolver
from mystic.termination import VTR
from pathos.pools import ProcessPool as Pool

# set the ranges
target = [2.9306538, 4.6817646, 3.6026807, 3.233392]
lower_bounds = [x * 0.95 for x in target]
upper_bounds = [x * 1.05 for x in target]

# set random seed so we can reproduce results
tools.random_seed(0)

# create a solver
solver = LatticeSolver(4, 8)

# set multi-processing pool
solver.SetMapper(Pool().map)

# since we have an search solver
# we specify what optimization algorithm to use within the search
# we tell the optimizer to not go more than 50 evaluations of our cost function
subsolver = NelderMeadSimplexSolver(4)
subsolver.SetEvaluationLimits(50, 50)
solver.SetNestedSolver(subsolver)

# set the range to search for all parameters
solver.SetStrictRanges(lower_bounds, upper_bounds)

# find the minimum
solver.Solve(CostFunction(4), VTR())

# print the best parameters
print(f"The best solution is {solver.bestSolution} with Rwp {solver.bestEnergy}")
print(f"The reference solutions is {target}")
ratios = [x / y for x, y in zip(target, solver.bestSolution)]
print(f"The ratios of to the reference values are {ratios}")

## An ensemble using only the cost function (changing the multi-processing pool and solver)

There are many things that can be changed. Here we give an example of changing from the multi-processing pool and the solver.

Above we ran using a ``ProcessPool``. We can also use another multi-processing capability the ``ThreadPool`` as well; however, since threads share memory we will have to rewrite the cost function slightly such that each call to ``CostFunction.function`` is indepedent.

In [ ]:
import MILK
import threading
import os
import shutil
import time
from mystic import models
from spotlight import filesystem

class CostFunction(models.AbstractFunction):

    def __init__(self, ndim=None, config=None):
        super().__init__(ndim=ndim)
        self.initialized = False
        self.config = config
        
    def function(self, p):

        t0 = time.time()

        # get editor and maudText
        editor = MILK.parameterEditor.editor()
        editor.parseConfig(self.config)
        maudText = MILK.maud.maudText()
        maudText.parseConfig(self.config)

        # set run dir based on process name
        editor.run_dirs = f"opt_{threading.current_thread().name}"
        maudText.run_dirs = editor.run_dirs

        # create run dir
        if os.path.exists(editor.run_dirs):
            shutil.rmtree(editor.run_dirs)
        filesystem.mkdir(editor.run_dirs)
        filesystem.cp(["/Users/cmbiwer/projects/milk_workshop/spotlight/docs/notebooks/sequential_refinement/templates/template_4768.par"], dest=editor.run_dirs) # CHANGE THIS LINE
        
        # set initial phase fractions
        editor.set_val(key='_pd_phase_atom_', value='0.33')
        editor.free(key='_pd_phase_atom_', wild=[0])

        # set lattice parameters as values from optimization
        editor.set_val(key='cell_length_a', sobj="alpha", value=str(p[0]))
        editor.set_val(key='cell_length_c', sobj="alpha", value=str(p[1]))
        editor.set_val(key='cell_length_a', sobj="steel", value=str(p[2]))
        editor.set_val(key='cell_length_a', sobj="beta", value=str(p[3]))

        # refine
        maudText.refinement(itr='1', ifile=editor.ifile, ofile=editor.ofile, simple_call=True)

        # get the statistic to return to the optimizer
        editor.get_val(key="_refine_ls_wR_factor_all")
        stat = float(editor.value[0])

        print(f"Our R-factor is {stat} and it took {time.time() - t0}s to compute")

        return stat

We can use the same ``config`` as defined in the prior examples. And now we can call the same optimization using a ``ThreadPool``. We can also use a different solver by using a different class. Above we used the ``LatticeSolver`` that samples a lattice. Below, we use the ``BuckshotSolver`` which performs random sampling to begin new optimizers.

In [ ]:
from mystic import tools
from mystic.solvers import LatticeSolver, BuckshotSolver
from mystic.solvers import NelderMeadSimplexSolver
from mystic.termination import VTR
from pathos.pools import ThreadPool as Pool

# set the ranges
target = [2.9306538, 4.6817646, 3.6026807, 3.233392]
lower_bounds = [x * 0.95 for x in target]
upper_bounds = [x * 1.05 for x in target]
    
# set random seed so we can reproduce results
tools.random_seed(0)
    
# create a solver
solver = BuckshotSolver(4, 8)
    
# set multi-processing pool
solver.SetMapper(Pool().map)
    
# since we have an search solver
# we specify what optimization algorithm to use within the search
# we tell the optimizer to not go more than 50 evaluations of our cost function
subsolver = NelderMeadSimplexSolver(4)
subsolver.SetEvaluationLimits(50, 50)
solver.SetNestedSolver(subsolver)
    
# set the range to search for all parameters
solver.SetStrictRanges(lower_bounds, upper_bounds)
    
# find the minimum
solver.Solve(CostFunction(4, config=config), VTR())
    
# print the best parameters
print(f"The best solution is {solver.bestSolution} with Rwp {solver.bestEnergy}")
print(f"The reference solutions is {target}")
ratios = [x / y for x, y in zip(target, solver.bestSolution)]
print(f"The ratios of to the reference values are {ratios}")

This concludes the whirlwind tutorial of applying ensembles of optimizers to Rietveld analysis.